In [1]:
import pandas as pd

pd.set_option('display.max_colwidth',100)

In [2]:
df = pd.read_csv('sample_text.csv')
df.shape

(8, 2)

In [3]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [4]:
# Converting text column of the df into vectors

from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)
vectors.shape

/home/computervision/Desktop/siddhant/my_work/llm_agents/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(8, 768)

In [5]:
dim = vectors.shape[1]
dim

768

In [7]:
import faiss

index = faiss.IndexFlatL2(dim)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f92719cc960> >

In [8]:
index.add(vectors)

Now we can use this index to query. For this first we have to provide search query and convert it into vector and then query.

In [19]:
search_query = 'i want to explore new things'
vec = encoder.encode(search_query)
vec.shape

(768,)

In [20]:
# Now index require 2D array so we will convert vec into 2D using numpy
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [21]:
distances, I = index.search(svec, k=2)

In [22]:
df.iloc[I[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


Now next step is to retreive relevant info from the chunks and provide it to LLM to make our predictions